Point Service
===========

The point service is the main database NF provides for finding and storing point-level metadata.  Using the points API, you can:

* Find and edit automation points
* Update metadata
* Delete points
* Retrieve timeseries data from points
* Subscribe to streaming updates from both data and points to view changes to the database.

In [2]:
import os
import grpc
import time
from normalgw.hpl import point_pb2, point_pb2_grpc

In [3]:
channel = grpc.insecure_channel("localhost:8080")
stub = point_pb2_grpc.PointManagerStub(channel)

GetPoints
--------

The `GetPoints` API provides paginated full-text search over your objects, based on RedisSearch.  

In [4]:
points = stub.GetPoints(point_pb2.GetPointsRequest(query="ACCUMULATOR", page_size=10))

DeletePoints
---------

`DeletePoints` removes all data from the database for the requested UUIDS, including time series data.  

Use this instead of editing Redis directly in order to update the distinct attribute sets and to propertly generate update log entries, so downstream services can be notified of object changes.

In [5]:
stub.DeletePoints(point_pb2.DeletePointsRequest(layer="hpl:bacnet:1", uuids=[p.uuid for p in points.points]))

GetDistinctAttrs
------

This API returns distinct values for attributes set on Point objects.  For instance, you can use this to retrieve the name of all objects in the database without needing to iterate over a large result set.   These sets are efficiently maintained in memory for fast querying.

In [6]:
stub.GetDistinctAttrs(point_pb2.GetDistinctAttrsRequest(attrs=["device_id"]))


attrs {
  key: "device_id"
}

GetData
------ 

GetData retrieves timeseries data from the database.  The amount of data available and retention period depend on the system configuration.

Currently the return type of all data is "double" due to limitations of RedisTimeseries; however this may change in the future.


In [6]:
# get a list of points you would like data for
points = stub.GetPoints(point_pb2.GetPointsRequest(query="ANALOG VALUE", page_size=10))
now = time.time()

# get data from the last hour, resampled to be every five minutes.
request = point_pb2.GetDataRequest(**{
    "layer": "hpl:bacnet:1",
    "uuids": [p.uuid for p in points.points],
    "window": {"seconds": 300},
    "from": {"seconds": int(now) - 3600},
    "to": {"seconds": int(now)},
    "method": "FIRST"
})
stub.GetData(request)

data {
  uuid: "adfb7c41-0909-3dad-9981-b54ae8c907e3"
  values {
    ts {
      seconds: 1624459500
    }
    double: 0.0
  }
  values {
    ts {
      seconds: 1624459800
    }
    double: 0.0
  }
  values {
    ts {
      seconds: 1624460100
    }
    double: 0.0
  }
  values {
    ts {
      seconds: 1624460400
    }
    double: 0.0
  }
}
data {
  uuid: "aa412cef-a8c4-31b6-ae4b-5a2e445ae5b5"
  values {
    ts {
      seconds: 1624459500
    }
    double: 0.0
  }
  values {
    ts {
      seconds: 1624459800
    }
    double: 0.0
  }
  values {
    ts {
      seconds: 1624460100
    }
    double: 0.0
  }
  values {
    ts {
      seconds: 1624460400
    }
    double: 0.0
  }
}
data {
  uuid: "cb06ed24-4a99-3e6c-acb9-4edb81468bdc"
  values {
    ts {
      seconds: 1624459500
    }
    double: 0.0
  }
  values {
    ts {
      seconds: 1624459800
    }
    double: 0.0
  }
  values {
    ts {
      seconds: 1624460100
    }
    double: 0.0
  }
  values {
    ts {
      seconds: 1624460